In [ ]:
import lib.facebook_scraper as fs
import lib.utils as utils
import pandas as pd
import numpy as np
import os
import time

In [ ]:
post_ids_df = pd.read_csv('data/vebay69/clean/post_ids.csv')
post_ids_df.head()

In [ ]:
fs.set_cookies('./cookies/cookie_1.txt')
# fs.enable_logging()

In [ ]:
NUMBER_POST = 10
post_list = []
sharer_list = []
total_sharer_real = 0
index_post_id_file_path = os.path.join('data','vebay69','url','i_post_id_share.txt')

In [ ]:
index_post_id = utils.read_index_file(file_path=index_post_id_file_path)
print(index_post_id)
posts_ids = utils.get_post_id_list(df=post_ids_df, start=index_post_id, length=NUMBER_POST)
posts_ids

In [ ]:
def extract_sharer(sharers, post_id):
    cnt_sharer = 0
    global sharer_list
    for sharer in sharers:
        sharer['post_id'] = post_id
        sharer_list.append(sharer)
        cnt_sharer += 1
        # time.sleep(np.random.uniform(0, 1))
    return cnt_sharer

In [ ]:
try:
    for post in fs.get_posts(
        post_urls=posts_ids,
        options={
            "allow_extra_requests": True,
            "sharers": True,
            "sharers": "generator",
        },
        timeout=120,
    ):
        print(post)
        post_list.append(post)
        num_sharer = extract_sharer(post['sharers'], post['original_request_url'])
        print(f'Extracted {num_sharer}/{post["shares"]} sharers')
        total_sharer_real += post['shares']
        utils.sleep(np.random.randint(5, 10))
        print()
except fs.exceptions.TemporarilyBanned:
    print("TEMPORARILY BANNED")
except fs.exceptions.AccountDisabled:
    print("ACCOUNT DISABLED")
except Exception as e:
    print(e)

In [ ]:
print(NUMBER_POST)
print(f"Số posts đã crawl: {len(post_list)}")
print(f"Số sharers đã crawl: {len(sharer_list)}/{total_sharer_real}")

In [ ]:
if sharer_list:
    utils.save_data(
        data_list=sharer_list,
        folder_path=os.path.join('data','vebay69','raw'),
        type="sharers"
    )
utils.write_index_file(
    file_path=index_post_id_file_path,
    index=index_post_id + len(post_list)
)